In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits


LVM_NAME = os.path.join("_dummy_data", "dummy-lvm-obs.fits.gz")
DESI_NAME = os.path.join("_dummy_data", "dummy-desi-obs.fits.gz")

In [2]:
lvm_fits = fits.open(LVM_NAME, memmap=False)
desi_fits = fits.open(DESI_NAME, memmap=False)

In [8]:
# TRANSLATOR CONFIGURATION GENERATOR
# lvmdrp/scripts/_gen_translator_configs.py

import collections


class OrderedSet(collections.Set):
    """A set without automatic sorting
    
    taken from: https://bit.ly/3AEtp9i"""
    def __init__(self, iterable=()):
        self.d = collections.OrderedDict.fromkeys(iterable)

    def __len__(self):
        return len(self.d)

    def __contains__(self, element):
        return element in self.d

    def __iter__(self):
        return iter(self.d)

def find_orphans(input_fits, header_ihdus_keys):
    """Return a dictionary with keys=(ihdu, key) and values='label' for missing cards in 'header_ihdus_keys'
    
    Parameters:
    -----------
    input_fits: astropy.io.fits.HDUList instance
        FITS file where to find orphan header cards
    header_ihdus_keys: list
        a list of tuples=(ihdu,key) for the reference header cards
    
    Returns:
    --------
    orphans: list
        list of orphan header keys
    """
    ihdus, keys = zip(*header_ihdus_keys)
    orphans = []
    for ihdu, lvm_hdu in enumerate(input_fits):
        hdu_mask = np.array(ihdus)==ihdu
        orphan_keys = OrderedSet(lvm_hdu.header.keys()) - OrderedSet(np.array(keys)[hdu_mask])
        orphans.extend(zip([ihdu]*len(orphan_keys), orphan_keys))
    return orphans

LVM2DESI_RAW_HEADER_MAP = {
    (0,"SIMPLE"): (1,"SIMPLE"),
    (0,"BITPIX"): (1,"BITPIX"),
    (0,"NAXIS"): (1,"NAXIS"),
    (0,"NAXIS1"): (1,"NAXIS1"),
    (0,"NAXIS2"): (1,"NAXIS2"),
    (0,"BSCALE"): (1,"BSCALE"),
    (0,"BZERO"): (1,"BZERO"),
    (0,"SPEC"): (1,"SPECGRPH"),
    (0,"OBSERVAT"): (0,"OBSERVAT"),
    (0,"OBSTIME"): (0,"DATE-OBS"),
    (0,"EXPTIME"): (1,"EXPTIME"),
    (0,"IMAGETYP"): (1,"FLAVOR"),
    (0,"INTSTART"): (1,"DATE-OBS"),
#     (0,"INTEND"): (1,"INTEND"),
#     (0,"BINNING"): (1,"BINNING"),
    (0,"CCD"): (1,"CAMERA"),
}
LVM2DESI_RAW_DATA_MAP = {
    0:1
}
LVM2DESI_RAW_HEADER_TRANS = {
    "SPEC": lambda v: int(v[-1]),
}
LVM_ORPHANS = find_orphans(input_fits=lvm_fits, header_ihdus_keys=LVM2DESI_RAW_HEADER_MAP.keys())
# LVM_ORPHANS

DESI_ORPHANS = find_orphans(input_fits=desi_fits, header_ihdus_keys=LVM2DESI_RAW_HEADER_MAP.values())
# DESI_ORPHANS

In [4]:
from copy import deepcopy as copy


def lvm2desi_translator(lvm_fits, desi_template, header_map, header_transformer, data_map):
    """Return a translated version of the 'lvm_fits' using 'desi_template'
    
    Parameters:
    -----------
    lvm_fits: astropy.io.fits.HDUList instance
        the target LVM FITS file to translate into DESI
    desi_template: astropy.io.fits.HDUList instance
        the DESI template to translate to
    header_map: dict
        a dictionary holding the mapping between LVM and DESI header cards
    header_translator: dict
        a dictionary with translation of data types from LVM to DESI
    data_map: dict
        a dictionary to translate data (e.g., image, cubes) from one header to other
    
    Returns:
    --------
    desi_fits: astropy.io.fits.HDUList instance
        the translated LVM to DESI FITS file
    """
    desi_fits = copy(desi_template)
    
    # update header taking into account transformers
    # TODO: update HDU names
    # TODO: update redundant information in several HDUs
    for (lvm_ihdu,lvm_key), (desi_ihdu,desi_key) in header_map.items():
        trans = header_transformer.get(lvm_key, lambda v: v)
        desi_fits[desi_ihdu].header[desi_key] = trans(lvm_fits[lvm_ihdu].header[lvm_key])
    
    # update data images/cubes
    # TODO: update prescan and overscan regions
    for lvm_ihdu, desi_ihdu in data_map.items():
        desi_fits[desi_ihdu].data = lvm_fits[lvm_ihdu].data
    
    # TODO: implement tables update

    return desi_fits

In [5]:
lvm_example_fits = fits.open("/disk-a/mejia/Research/UNAM/lvm-drp/data/data.sdss5.org/sas/sdsswork/data/lvm/lco/test/59365/sdR-s-r1-00000320.fits.gz")

lvm_example_trans = lvm2desi_translator(
    lvm_fits=lvm_example_fits,
    desi_template=desi_fits,
    header_map=LVM2DESI_RAW_HEADER_MAP,
    data_map=LVM2DESI_RAW_DATA_MAP,
    header_transformer=LVM2DESI_RAW_HEADER_TRANS
)
lvm_example_trans.info()

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  SPS           1 PrimaryHDU     165   ()      
  1  Z0            1 ImageHDU       138   (4120, 4080)   uint16   
  2  SPECTCONS     1 BinTableHDU    100   1R x 35C   ['J', '26A', '15A', 'E', '12A', '19A', '5A', '4A', '3A', '4A', '3A', '5A', '4A', '8A', '2A', '6A', '8A', '2A', '8A', 'E', 'E', 'E', 'E', 'E', 'E', '3A', '3A', '3A', '3A', '7A', '7A', '7A', '3A', '3A', '3A']   
